In [5]:
import pyspark
from pyspark.sql import SparkSession
import os

NESSIE_URI = os.getenv("NESSIE_URI")
MINIO_HOST = os.getenv("MINIO_HOST")
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY")
MINIO_SECRET_KEY =os.getenv("MINIO_SECRET_KEY")

jar_packages = [
  "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
  "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.95.0",
  "software.amazon.awssdk:bundle:2.17.178",
  "software.amazon.awssdk:url-connection-client:2.17.178"
]

spark_extensions = [
  "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
  "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
]

# fs.s3a.path.style.access true
# dremio.s3.compat true
# fs.s3a.endpoint http://minio:9000

conf = (
  pyspark.SparkConf()
    .setAppName('iceberg_hello_world')
    .set('spark.jars.packages', ','.join(jar_packages))
	.set('spark.sql.extensions', ','.join(spark_extensions))
	.set('spark.sql.catalog.nessie', "org.apache.iceberg.spark.SparkCatalog")
	.set('spark.sql.catalog.nessie.uri', NESSIE_URI)
	.set('spark.sql.catalog.nessie.ref', 'main')
	.set('spark.sql.catalog.nessie.authentication.type', 'NONE')
	.set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
	.set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
	.set('spark.sql.catalog.nessie.s3.endpoint', MINIO_HOST)
	.set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
	.set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
	.set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
	.set('spark.hadoop.fs.s3a.path.style.access', 'true')
  .set('spark.hadoop.fs.s3a.connection.timeout', '600000')
	.set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
	.set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
	.set('spark.hadoop.fs.s3a.endpoint', MINIO_HOST)		
)

# .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
# .config("spark.hadoop.fs.s3a.access.key", "SEU_ACCESS_KEY") \
# .config("spark.hadoop.fs.s3a.secret.key", "SEU_SECRET_KEY") \
# .config("spark.hadoop.fs.s3a.endpoint", "http://SEU_MINIO_ENDPOINT:9000") \
# .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    
## Start Spark Session
spark = SparkSession.builder.config(conf=conf).master("local").getOrCreate()
print("Spark Running")

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson2 USING iceberg AS SELECT * FROM csv_open_2023").show()
spark.sql("SELECT COUNT(*) FROM nessie.df_open_2023_lesson2").show()

24/09/28 01:32:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Spark Running


24/09/28 01:32:25 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.18.0.2 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/app/datasets/df_open_2023.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expression

Py4JJavaError: An error occurred while calling o108.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.18.0.2 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/app/datasets/df_open_2023.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon

In [7]:
spark.sql("SHOW TABLES IN nessie").show()

# 

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|         |df_open_2023_lesson2|      false|
+---------+--------------------+-----------+



In [3]:
## QUERY THE COUNT OF ENTRIES
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

+------+
| Total|
+------+
|280003|
+------+

